# Homework 4: Implementing a Text Generation Model

Student: Georgy Andryushchenko

In [1]:
!pip install datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.

In [13]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 3.4 MB/s eta 0:00:00


## 1. Loading the Model

* Load the model with specific configurations, choose the model.
* Load the corresponding tokenizer.

In [1]:
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Load the corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## 2. Freezing Model Parameters

* Freeze all model parameters to prevent them from being updated
during training.
* Cast the small parameters (e.g., layer normalization parameters) to
fp32 for stability

In [2]:
# Freeze all model parameters
for param in model.parameters():
    param.requires_grad = False

# Cast layer normalization parameters to fp32 for stability
for name, param in model.named_parameters():
    if 'ln' in name:
        param.data = param.data.float()

## 3. Enable Gradient Checkpointing

* Enable gradient checkpointing to reduce the number of stored activations.
* Enable input requires gradients.

In [3]:
# Enable gradient checkpointing
model.gradient_checkpointing_enable()

# Enable input requires gradients
model.transformer.wte.weight.requires_grad = True
model.transformer.wpe.weight.requires_grad = True

## 4. Custom Output Casting

* Create a custom class to cast the output of lm head to float32.
* Replace the model’s lm head with an instance of this custom class.

In [4]:
import torch
from torch import nn

# Custom class to cast the output of lm head to float32
class CustomLMHead(nn.Module):
    def __init__(self, original_lm_head):
        super(CustomLMHead, self).__init__()
        self.original_lm_head = original_lm_head

    def forward(self, hidden_states):
        output = self.original_lm_head(hidden_states)
        return output.to(torch.float32)

# Replace the model’s lm head with an instance of this custom class
model.lm_head = CustomLMHead(model.lm_head)

## 5. Helper Function

* Implement a helper function to print the number of trainable parameters in the model.

In [5]:
def print_trainable_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable_params}")
    print(f"Total parameters: {total_params}")
    print(f"Percentage of trainable parameters: {100 * trainable_params / total_params:.2f}%")

# Example usage with the GPT-2 model
print_trainable_parameters(model)

Trainable parameters: 39383808
Total parameters: 124439808
Percentage of trainable parameters: 31.65%


## 6. Prompt Creation

* Implement a function to format context, question, and answer into a
prompt template.
* Map the QA dataset to this prompt format using the tokenizer.

In [6]:
def create_prompt(context, question, answer=None):
    if answer:
        return f"Context: {context}\nQuestion: {question}\nAnswer: {answer}{tokenizer.special_tokens_map['eos_token']}"
    else:
        return f"Context: {context}\nQuestion: {question}\nAnswer:"

# Example usage
context = "GPT-2 is a large transformer-based language model developed by OpenAI."
question = "What is GPT-2?"
answer = "A large transformer-based language model developed by OpenAI."
prompt = create_prompt(context, question, answer)
print(prompt)


Context: GPT-2 is a large transformer-based language model developed by OpenAI.
Question: What is GPT-2?
Answer: A large transformer-based language model developed by OpenAI.<|endoftext|>


In [7]:
from datasets import load_dataset

# Load the SQuAD dataset
dataset = load_dataset('squad')

In [8]:
from tqdm import tqdm

def tokenize_prompts(examples, tokenizer, max_length=512):
    prompts = []
    for ex in tqdm(examples):
        context = ex['context']
        question = ex['question']
        answer = ex['answers']['text'][0]
        prompt = create_prompt(context, question, answer)
        tokenized = tokenizer(prompt, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
        prompts.append(tokenized)
    return prompts

# Apply the function to the SQuAD dataset
train_data = dataset['train']
tokenized_train_data = tokenize_prompts(train_data, tokenizer)

100%|██████████| 87599/87599 [01:40<00:00, 874.87it/s]


## 7. Training the Model
* Train the model

In [9]:
# Convert the list of dictionaries to a format suitable for the Trainer
def format_for_trainer(tokenized_data):
    input_ids = [item['input_ids'] for item in tokenized_data]
    attention_masks = [item['attention_mask'] for item in tokenized_data]
    return {'input_ids': input_ids, 'attention_mask': attention_masks}

train_dataset = format_for_trainer(tokenized_train_data)

In [10]:
# Define a custom dataset class for the Trainer
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.data['input_ids'][idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.data['attention_mask'][idx], dtype=torch.long)
        }

# Create a custom dataset
train_dataset = CustomDataset(train_dataset)

In [11]:
!rm -rf model/

In [13]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
        output_dir='./output_dir',
        overwrite_output_dir=True,
        per_device_train_batch_size=16,
        max_steps=100,
        logging_steps=10
    )

trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
)

trainer.train()

model.save_pretrained('model')

max_steps is given, it will override any value given in num_train_epochs
<ipython-input-10-134f89a0a66f>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(self.data['input_ids'][idx], dtype=torch.long),
<ipython-input-10-134f89a0a66f>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(self.data['attention_mask'][idx], dtype=torch.long)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1,3.569700
2,3.700300
3,3.643200
4,3.511600
5,3.684300
6,3.448900
7,3.520700
8,3.503100
9,3.778000
10,3.600900


## 8. Loading LoRA Model

* Load the configuration for the LoRA model.
* Load the base model and tokenizer.
* Load the LoRA model using the pre-trained parameters.


In [14]:
from peft import LoraConfig, PeftModel
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Define the LoRA configuration
lora_config = LoraConfig(
    r=8,  # Rank of the LoRA matrices
    lora_alpha=16,  # Scaling factor for the LoRA matrices
)


In [15]:
# Load the base model and tokenizer
# base_model_name = 'model'
model = GPT2LMHeadModel.from_pretrained('model')

In [16]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [17]:
# Apply the LoRA configuration to the model
from peft import LoraConfig, PeftModel, get_peft_model

lora_model = get_peft_model(model, lora_config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


## 9. Inference Function

* Implement an inference function to generate answers based on a given
context and question

In [27]:
def inference(model, tokenizer, context, question, max_length=512):
    # Create the prompt
    prompt = create_prompt(context, question)

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors='pt')

    # Generate the answer
    outputs = lora_model.generate(
        **inputs,
        pad_token_id=tokenizer.eos_token_id,
        max_length=max_length,
    )

    # Decode the generated tokens
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer

## 10. Test Inference
* Test the inference function with sample contexts and questions.

In [28]:
for ex in dataset['validation']:
    answer = inference(lora_model, tokenizer, ex['context'], ex['question'])
    print(answer)
    break

Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Question: Which NFL team represented the AFC at Super Bowl 50?
Answer: The New England Patriots, who won the Super Bowl in Super Bowl 50, were the first team to win the Super Bowl in the history of the league. The N